## _*Using Qiskit Aqua for the vertex-cover problems*_

A vertex cover of a graph is a set of vertices such that each edge of the graph is incident to at least one vertex of the set. The goal of NPC problem is to minimize the size of the vertex cover. 

We will go through two examples to show:
1. How to run the optimization
2. How how to run the optimization with the VQE.

#### The problem and the brute-force method

The problem is as follows. the graph is in the adjacent matrix form.

In [1]:
import numpy as np
from qiskit import BasicAer
from qiskit.optimization.applications.ising import vertex_cover
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.applications.ising.common import random_graph, sample_most_likely

np.random.seed(100)
num_nodes = 3
w = random_graph(num_nodes, edge_prob=0.8, weight_range=10)
print(w)

[[ 0.  5. -4.]
 [ 5.  0.  0.]
 [-4.  0.  0.]]


The brute-force method is as follows. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a vertex is either 0 (meaning the vertex is not in the cover) or 1 (meaning the vertex is in the cover). We print the binary assignment that satisfies the definition of the vertex cover and corresponds to the minimial size. 

In [2]:
def brute_force():
    # brute-force way
    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = num_nodes
    max = 2**L
    minimal_v = np.inf
    for i in range(max):
        cur = bitfield(i, L)

        cur_v = vertex_cover.check_full_edge_coverage(np.array(cur), w)
        if cur_v:
            nonzerocount = np.count_nonzero(cur)
            if nonzerocount < minimal_v:
                minimal_v = nonzerocount

    return minimal_v

size = brute_force()
print('Size of the vertex cover', size)

Size of the vertex cover 1


In [3]:
qubit_op, offset = vertex_cover.get_operator(w)

#### Part I: Run the optimization

In [4]:
algo = NumPyMinimumEigensolver(qubit_op)
result = algo.run()

x = sample_most_likely(result.eigenstate)
sol = vertex_cover.get_graph_solution(x)
print(sol)
print('Size of the vertex cover', np.count_nonzero(sol))

[1 0 0]
Size of the vertex cover 1


#### Part II: Run the optimization with the VQE

In [5]:
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal

aqua_globals.random_seed = 100

optimizer = SPSA(maxiter=200)
var_form = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cz', reps=3)
vqe = VQE(qubit_op, var_form, optimizer)

backend = BasicAer.get_backend('qasm_simulator')
result = vqe.run(backend)

x = sample_most_likely(result.eigenstate)
sol = vertex_cover.get_graph_solution(x)
print('Size of the vertex cover', np.count_nonzero(sol))

Size of the vertex cover 1
